# Build the logbook for Hologram with empty or filters

- author : Sylvie Dagoret-Campagne
- affiliation : IJCLab/IN2P3/CNES, DESC-LSST fullmember, behalf AuxTel Teal VERA C. Rubin observatory
- creation date : September 6th 2023
- last update : September 20th 2023


Works directory per directory (day by day)

For USDF

In [1]:
import sys
sys.path

['/sdf/data/rubin/user/dagoret/AuxTel_summit_tests_2023/runspectractor_standalone/2023_09',
 '/home/d/dagoret/repos/repos_w_2023_35/analysis_tools/python',
 '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-7.0.1/Linux64/cp_pipe/g857a3418e4+7df23b908a/python',
 '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-7.0.1/Linux64/shapelet/g44018dc512+ea6441928c/python',
 '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-7.0.1/Linux64/meas_modelfit/g93c4d6e787+79b822255a/python',
 '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-7.0.1/Linux64/scarlet_extensions/g9d18589735+d445b4966c/lib/python',
 '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-7.0.1/Linux64/proxmin/g33b4157f25/lib/python',
 '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-7.0.1/Linux64/scarlet/g63ff1f842d+4083830bf8/lib/python',
 '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-7.0.1/Linux64/meas_extensions_scarlet/g943cb464a2+f79590064d/python',
 '/opt/lsst/software/stack/stack/miniconda3-py

In [2]:
! ls /home/d/dagoret/rubin-user/AuxTelComm/notebooks_usdf/GeneratepseudoPostISRCCD/my_postisrccd_img

ls: cannot access /home/d/dagoret/rubin-user/AuxTelComm/notebooks_usdf/GeneratepseudoPostISRCCD/my_postisrccd_img: No such file or directory


In [3]:
!ls /home/d/dagoret/rubin-user/DATA/AuxtelData2023/my_postisrccd_img_forspectractor_2023/empty~holo4_003


20230117  20230131  20230705  20230720	20230815  20230913
20230118  20230202  20230706  20230801	20230817  20230914
20230119  20230704  20230718  20230802	20230912


# Select the combination DATE, FILTER, DISPERSER

In [4]:
DATE = '20230912'
filterdispersernames = ["empty~holo4_001",\
                        "empty~holo4_003",\
                        "collimator~holo4_003",\
                        "cyl_lens~holo4_001",\
                        "cyl_lens~holo4_003",\
                       ]
filterdispersername = filterdispersernames[2]
version="v7.0"  # new orientation

# If position need to be specified add it in this dictionnary

In [5]:
# reconstructed / good /xpos /ypos

if version == "v7.0":
    info={\

          # with red filter
          'exposure_2023031600351_postisrccd.fits':(1,1,300.,1700),\
         }
        

# Imports

In [6]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from mpl_toolkits.axes_grid1 import make_axes_locatable
import pandas as pd

import matplotlib.ticker                         # here's where the formatter is
import os
import re

plt.rcParams["figure.figsize"] = (16,4)
plt.rcParams["axes.labelsize"] = 'xx-large'
plt.rcParams['axes.titlesize'] = 'xx-large'
plt.rcParams['xtick.labelsize']= 'xx-large'
plt.rcParams['ytick.labelsize']= 'xx-large'

In [7]:
from astropy.io import fits

In [8]:
import imp
MODULE_EXTENSIONS = ('.py', '.pyc', '.pyo')

/tmp/ipykernel_4742/3547059130.py:1: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  import imp


In [9]:
def package_contents(package_name):
    file, pathname, description = imp.find_module(package_name)
    if file:
        raise ImportError('Not a package: %r', package_name)
    # Use a set because some may be both source and compiled.
    return set([os.path.splitext(module)[0]
        for module in os.listdir(pathname)
        if module.endswith(MODULE_EXTENSIONS)])

In [10]:
package_contents("spectractor")

{'__init__',
 '_version',
 'astrometry',
 'config',
 'logbook',
 'parameters',
 'tools'}

In [11]:
package_contents("spectractor/extractor")

{'__init__',
 'background',
 'chromaticpsf',
 'dispersers',
 'extractor',
 'images',
 'psf',
 'spectroscopy',
 'spectrum',
 'targets'}

# Initialisation

In [12]:
dir_logbooks="logbooks"

In [13]:
import os
if not os.path.exists(dir_logbooks):
    os.makedirs(dir_logbooks)

## Get inputs files and sort them according date and number

In [14]:
home=os.getenv("HOME")

In [15]:
image_path=filterdispersername
logbook_csv=os.path.join(dir_logbooks,"auxtellogbook_" + filterdispersername + "_"+ DATE +"_" + version + ".csv")
logbook_xlsx=os.path.join(dir_logbooks,"auxtellogbook_" + filterdispersername + "_" +DATE +"_" + version + ".xlsx")

# Path

In [16]:
def GetPaths(computer_name):
    """
    """
    
    if computer_name == "HOSTCC":
        
        path_auxtel="/sps/lsst/groups/auxtel"
        path_spectractor=os.path.join(path_auxtel,"softs/github/desc/Spectractor")
        path_spectractor_config=os.path.join(path_spectractor,"config")
    
        path_images=os.path.join(path_auxtel,"data/hack_usdf/my_postisrccd_img_forspectractor_2023/"+filterdispersername+"/"+DATE)
        if configdir == "":
            path_output_spectractor=os.path.join(path_auxtel,"data/2023/OutputSpectractor/"+imagemode+"/"+filterdispersername+"/"+DATE)
        else:
            path_output_spectractor=os.path.join(path_auxtel,"data/2023/OutputSpectractor/"+imagemode+"/"+filterdispersername+"/"+configdir+"/"+DATE)
            path_topoutput_spectractor=os.path.join(path_auxtel,"data/2023/OutputSpectractor/"+imagemode+"/"+filterdispersername+"/"+configdir)
            path_toptopoutput_spectractor=os.path.join(path_auxtel,"data/2023/OutputSpectractor/"+imagemode+"/"+filterdispersername)
            path_toptoptopoutput_spectractor=os.path.join(path_auxtel,"data/2023/OutputSpectractor/"+imagemode)
        
    elif computer_name == "LAPTOP1":
        path_auxtel="/Users/dagoret/DATA/AuxTelData2023"
        path_spectractor=os.path.join(path_auxtel,"/Users/dagoret/MacOSX/GitHub/LSST/Spectractor")
        path_spectractor_config=os.path.join(path_spectractor,"config")
        path_images=os.path.join(path_auxtel,"data/2023/"+filterdispersername+"/"+DATE)
        if configdir == "":
            path_output_spectractor=os.path.join(path_auxtel,"data/2023/OutputSpectractor/"+imagemode+"/"+filterdispersername+"/"+DATE)
        else:
            path_output_spectractor=os.path.join(path_auxtel,"data/2023/OutputSpectractor/"+imagemode+"/"+filterdispersername+"/"+configdir+"/"+DATE)
            path_topoutput_spectractor=os.path.join(path_auxtel,"data/2023/OutputSpectractor/"+imagemode+"/"+filterdispersername+"/"+configdir)
            path_toptopoutput_spectractor=os.path.join(path_auxtel,"data/2023/OutputSpectractor/"+imagemode+"/"+filterdispersername)
            path_toptoptopoutput_spectractor=os.path.join(path_auxtel,"data/2023/OutputSpectractor/"+imagemode)
        
        
    elif computer_name == "WORKIMAC":
        path_auxtel="/Users/sylvie/DATA/AuxTelData2023"
        path_spectractor=os.path.join(path_auxtel,"/Users/sylvie/MacOSX/GitHub/LSST/Spectractor")
        path_spectractor_config=os.path.join(path_spectractor,"config")
        path_images=os.path.join(path_auxtel,"data/2023/"+filterdispersername+"/"+DATE)
        if configdir == "":
            path_output_spectractor=os.path.join(path_auxtel,"data/2023/OutputSpectractor/"+imagemode+"/"+filterdispersername+"/"+DATE)
        else:
            path_output_spectractor=os.path.join(path_auxtel,"data/2023/OutputSpectractor/"+imagemode+"/"+filterdispersername+"/"+configdir+"/"+DATE)
            path_topoutput_spectractor=os.path.join(path_auxtel,"data/2023/OutputSpectractor/"+imagemode+"/"+filterdispersername+"/"+configdir)
            path_toptopoutput_spectractor=os.path.join(path_auxtel,"data/2023/OutputSpectractor/"+imagemode+"/"+filterdispersername)
            path_toptoptopoutput_spectractor=os.path.join(path_auxtel,"data/2023/OutputSpectractor/"+imagemode)
            
            
        
    elif computer_name == "HOMEIMAC":
        path_auxtel="/Volumes/Backup2020/DATA"
        path_spectractor=os.path.join(path_auxtel,"MacOSX/GitHub/LSST/Spectractor")
        path_spectractor_config=os.path.join(".","config")
        path_images=os.path.join(path_auxtel,"hack_usdf/my_postisrccd_img_forspectractor_2023/"+filterdispersername+"/"+DATE)
        
        # /Volumes/Backup2020/DATA/AuxTelDATA2023/data/2023/OutputSpectractor
        
        if configdir == "":
            path_output_spectractor=os.path.join(path_auxtel,"AuxTelDATA2023/data/2023/OutputSpectractor/"+imagemode+"/"+filterdispersername+"/"+DATE)
        else:
            path_output_spectractor=os.path.join(path_auxtel,"AuxTelDATA2023/data/2023/OutputSpectractor/"+imagemode+"/"+filterdispersername+"/"+configdir+"/"+DATE)
            path_topoutput_spectractor=os.path.join(path_auxtel,"AuxTelDATA2023/data/2023/OutputSpectractor/"+imagemode+"/"+filterdispersername+"/"+configdir)
            path_toptopoutput_spectractor=os.path.join(path_auxtel,"AuxTelDATA2023/data/2023/OutputSpectractor/"+imagemode+"/"+filterdispersername)
            path_toptoptopoutput_spectractor=os.path.join(path_auxtel,"AuxTelDATA2023/data/2023/OutputSpectractor/"+imagemode)
    # /home/d/dagoret/rubin-user/DATA/AuxtelData2023/my_postisrccd_img_forspectractor_2023/empty~holo4_003
    elif computer_name == "USDF":
        path_auxtel="/home/d/dagoret/rubin-user/DATA"
        path_spectractor=os.path.join("/home/d/dagoret","repos/repos_w_2023_35/Spectractor")
        path_spectractor_config=os.path.join(".","config")
        #path_images=os.path.join(path_auxtel,"AuxtelData2023/my_postisrccd_img_forspectractor_2023/"+filterdispersername+"/"+DATE)
        path_images=os.path.join(path_auxtel,"AuxtelData2023/dm_postisrccd_img_forspectractor_2023/"+filterdispersername+"/"+DATE)
        
        # /Volumes/Backup2020/DATA/AuxTelDATA2023/data/2023/OutputSpectractor
        
        if configdir == "":
            path_output_spectractor=os.path.join(path_auxtel,"AuxtelData2023/OutputSpectractor/"+imagemode+"/"+filterdispersername+"/"+DATE)
        else:
            path_output_spectractor=os.path.join(path_auxtel,"AuxTelData2023/OutputSpectractor/"+imagemode+"/"+filterdispersername+"/"+configdir+"/"+DATE)
            path_topoutput_spectractor=os.path.join(path_auxtel,"AuxtelData2023/OutputSpectractor/"+imagemode+"/"+filterdispersername+"/"+configdir)
            path_toptopoutput_spectractor=os.path.join(path_auxtel,"AuxtelData2023/OutputSpectractor/"+imagemode+"/"+filterdispersername)
            path_toptoptopoutput_spectractor=os.path.join(path_auxtel,"AuxtelData2023/OutputSpectractor/"+imagemode)
        
        
        
    else:
        print(f"Unknown computer {computer_name}")
        
              
    return path_auxtel,path_spectractor,path_spectractor_config,path_images,path_toptoptopoutput_spectractor, path_toptopoutput_spectractor, path_topoutput_spectractor, path_output_spectractor
        

In [17]:
configdir = " "
imagemode = " "
mycomputer_name = "USDF"
path_auxtel,path_spectractor,path_spectractor_config,path_images,path_toptoptopoutput_spectractor, path_toptopoutput_spectractor, path_topoutput_spectractor, path_output_spectractor  = GetPaths(mycomputer_name)

In [18]:
path_auxtel

'/home/d/dagoret/rubin-user/DATA'

In [19]:
#  /home/d/dagoret/rubin-user/DATA/AuxtelData2023/my_postisrccd_img_forspectractor_2023/empty~holo4_003
path_images

'/home/d/dagoret/rubin-user/DATA/AuxtelData2023/dm_postisrccd_img_forspectractor_2023/collimator~holo4_003/20230912'

# Check the images exists

In [20]:
dir_images=path_images

In [21]:
dir_images

'/home/d/dagoret/rubin-user/DATA/AuxtelData2023/dm_postisrccd_img_forspectractor_2023/collimator~holo4_003/20230912'

In [22]:
!ls /home/d/dagoret/rubin-user/DATA/AuxtelData2023/my_postisrccd_img_forspectractor_2023

collimator~holo4_003  cyl_lens~holo4_003  empty~holo4_003
cyl_lens~holo4_001    empty~holo4_001	  transmitdata.sh


In [23]:
! ls /home/d/dagoret/rubin-user/DATA/AuxtelData2023/my_postisrccd_img_forspectractor_2023/empty~holo4_003/20230815

exposure_2023081500089_pseudo-postisrccd.fits
exposure_2023081500092_pseudo-postisrccd.fits
exposure_2023081500093_pseudo-postisrccd.fits


In [24]:
all_files=os.listdir(dir_images)

In [25]:
all_files

['exposure_2023091200282_dmpostisrccd.fits',
 'exposure_2023091200386_dmpostisrccd.fits',
 'exposure_2023091200251_dmpostisrccd.fits',
 'exposure_2023091200401_dmpostisrccd.fits',
 'exposure_2023091200426_dmpostisrccd.fits',
 'exposure_2023091200296_dmpostisrccd.fits',
 'exposure_2023091200297_dmpostisrccd.fits',
 'exposure_2023091200281_dmpostisrccd.fits',
 'exposure_2023091200311_dmpostisrccd.fits',
 'exposure_2023091200341_dmpostisrccd.fits',
 'exposure_2023091200402_dmpostisrccd.fits',
 'exposure_2023091200372_dmpostisrccd.fits',
 'exposure_2023091200371_dmpostisrccd.fits',
 'exposure_2023091200326_dmpostisrccd.fits',
 'exposure_2023091200266_dmpostisrccd.fits',
 'exposure_2023091200425_dmpostisrccd.fits',
 'exposure_2023091200252_dmpostisrccd.fits',
 'exposure_2023091200342_dmpostisrccd.fits',
 'exposure_2023091200387_dmpostisrccd.fits',
 'exposure_2023091200312_dmpostisrccd.fits',
 'exposure_2023091200356_dmpostisrccd.fits',
 'exposure_2023091200327_dmpostisrccd.fits',
 'exposure

In [26]:
all_selected_files = []
for filename in all_files:
    if re.search(".fits$",filename):
        all_selected_files.append(filename)
N=len(all_selected_files)

In [27]:
all_selected_files=np.array(all_selected_files)

### Sort files

In [31]:
def file_tag_forsorting(filename):
    #m=re.findall('exposure.*_(.*)_pseudo-postisrccd[.]fits$',filename)
    m=re.findall('exposure.*_(.*)_dmpostisrccd[.]fits$',filename)
    
    return m[0]

In [29]:
file_tag_forsorting('exposure_2022031700310_pseudo-postisrccd.fits')

'2022031700310'

In [32]:
#all_tags = [ file_tag_forsorting(filename)[0] for filename in all_selected_files]
all_tags = [ file_tag_forsorting(filename) for filename in all_selected_files]
all_tags=np.array(all_tags)
all_tags

array(['2023091200282', '2023091200386', '2023091200251', '2023091200401',
       '2023091200426', '2023091200296', '2023091200297', '2023091200281',
       '2023091200311', '2023091200341', '2023091200402', '2023091200372',
       '2023091200371', '2023091200326', '2023091200266', '2023091200425',
       '2023091200252', '2023091200342', '2023091200387', '2023091200312',
       '2023091200356', '2023091200327', '2023091200267', '2023091200357'],
      dtype='<U13')

In [33]:
indexes=np.argsort(all_tags)

In [34]:
sorted_files=all_selected_files[indexes]

In [35]:
sorted_files

array(['exposure_2023091200251_dmpostisrccd.fits',
       'exposure_2023091200252_dmpostisrccd.fits',
       'exposure_2023091200266_dmpostisrccd.fits',
       'exposure_2023091200267_dmpostisrccd.fits',
       'exposure_2023091200281_dmpostisrccd.fits',
       'exposure_2023091200282_dmpostisrccd.fits',
       'exposure_2023091200296_dmpostisrccd.fits',
       'exposure_2023091200297_dmpostisrccd.fits',
       'exposure_2023091200311_dmpostisrccd.fits',
       'exposure_2023091200312_dmpostisrccd.fits',
       'exposure_2023091200326_dmpostisrccd.fits',
       'exposure_2023091200327_dmpostisrccd.fits',
       'exposure_2023091200341_dmpostisrccd.fits',
       'exposure_2023091200342_dmpostisrccd.fits',
       'exposure_2023091200356_dmpostisrccd.fits',
       'exposure_2023091200357_dmpostisrccd.fits',
       'exposure_2023091200371_dmpostisrccd.fits',
       'exposure_2023091200372_dmpostisrccd.fits',
       'exposure_2023091200386_dmpostisrccd.fits',
       'exposure_2023091200387_

In [36]:
N=len(sorted_files)

# Build the logbook from the headers

In [37]:
idx=0
#ccdir='/sps/lsst/groups/auxtel/data/2021/holo/FlipCleans'
ccdir=dir_images
all_dates=[]
all_files=[]
all_seqnum=[]
all_subdir=[]
all_indexes=[]
all_targets=[]
all_filters=[]
all_disperser=[]
all_airmass=[]
all_exposures=[]
all_focus=[]
all_seeing=[]
all_P=[]
all_T=[]
all_H=[]
all_W=[]
for filename in sorted_files:
    fullfilename=os.path.join(dir_images,filename)
    hdu = fits.open(fullfilename)
    hdu0=hdu[0]
    hdr0=hdu0.header 
    all_indexes.append(idx)
    all_seqnum.append(hdr0['SEQNUM'])
    all_files.append(filename)
    all_subdir.append(ccdir)
    targetname=hdr0['OBJECT']
    all_targets.append(targetname)
    all_dates.append(hdr0['DATE-OBS'])
    all_airmass.append(hdr0['AMSTART'])
    all_filters.append(hdr0['FILTER'])
    all_disperser.append(hdr0['GRATING'])
    all_exposures.append(hdr0['EXPTIME'])
    all_focus.append(hdr0['FOCUSZ'])
    all_seeing.append(hdr0['SEEING'])
    all_P.append(hdr0['PRESSURE'])
    all_T.append(hdr0['AIRTEMP'])
    all_H.append(hdr0['HUMIDITY'])
    all_W.append(hdr0['WINDSPD'])
       
    
    idx+=1
    

# Make pandas dataframe

## Create pandas dataframe

In [38]:
df=pd.DataFrame()

In [39]:
len(all_airmass)

24

In [40]:
N=len(all_indexes)

## Fill with container info

In [41]:
df['index']=all_indexes
df['date']=all_dates
df['path']=all_subdir
df['file']=all_files
df['num']=all_seqnum
df['object']=all_targets
df['filter']=all_filters
df['disperser']=all_disperser
df['airmass']=all_airmass
df['exposure']=all_exposures
df['focus']=all_focus
df['seeing']=all_seeing
df['pressure']=all_P
df['temperature']=all_T
df['humidity']=all_H
df['windspd']=all_W

## Add dictionnary info

### Initialise new columns

In [42]:
Obj_posXpix=np.full(N,np.nan)
Obj_posYpix=np.full(N,np.nan)
Dx=np.full(N,np.nan)
Dy=np.full(N,np.nan)
run=np.full(N,np.nan)
quality=np.full(N,np.nan)
skip=np.full(N,"    ")

### Fill dictionnary info

In [43]:
# loop on rows index
for idx in np.arange(N):
    filename=all_files[idx]
    # check if filename is in dictionnary
    if filename in info.keys():
        (run_flag,quality_flag,xpos,ypos)=info[filename]
        Obj_posXpix[idx]=int(xpos)
        Obj_posYpix[idx]=int(ypos)
        run[idx]=run_flag
        if quality_flag==1:
            skip[idx] = "    "
            quality[idx]=1
        else:
            skip[idx] = "skip"
            quality[idx]=0

In [44]:
df['Obj-posXpix']=Obj_posXpix
df['Obj-posYpix']=Obj_posYpix
df['Dx']=Dx
df['Dy']=Dy
df['skip']=skip
df['run']=run
df['quality']=quality

In [45]:
pd.set_option('display.max_rows', None)

In [46]:
df

,index,date,path,file,num,object,filter,disperser,airmass,exposure,...,temperature,humidity,windspd,Obj-posXpix,Obj-posYpix,Dx,Dy,skip,run,quality
0,0,2023-09-13T02:49:39.056,/home/d/dagoret/rubin-user/DATA/AuxtelData2023...,exposure_2023091200251_dmpostisrccd.fits,251,HD167060,collimator,holo4_003,1.394863,30.0,...,-0.6000,74.775002,6.0882,NaN,NaN,NaN,NaN,,NaN,NaN
1,1,2023-09-13T02:50:21.993,/home/d/dagoret/rubin-user/DATA/AuxtelData2023...,exposure_2023091200252_dmpostisrccd.fits,252,HD167060,collimator,holo4_003,1.396716,30.0,...,-0.6000,73.375000,6.1716,NaN,NaN,NaN,NaN,,NaN,NaN
2,2,2023-09-13T03:00:34.619,/home/d/dagoret/rubin-user/DATA/AuxtelData2023...,exposure_2023091200266_dmpostisrccd.fits,266,Feige110,collimator,holo4_003,1.204973,30.0,...,-0.6375,77.387497,6.4218,NaN,NaN,NaN,NaN,,NaN,NaN
3,3,2023-09-13T03:01:17.817,/home/d/dagoret/rubin-user/DATA/AuxtelData2023...,exposure_2023091200267_dmpostisrccd.fits,267,Feige110,collimator,holo4_003,1.203372,30.0,...,-0.6000,77.324997,5.7546,NaN,NaN,NaN,NaN,,NaN,NaN
4,4,2023-09-13T03:10:24.382,/home/d/dagoret/rubin-user/DATA/AuxtelData2023...,exposure_2023091200281_dmpostisrccd.fits,281,HD009051,collimator,holo4_003,1.488436,30.0,...,-0.5250,73.750000,6.4218,NaN,NaN,NaN,NaN,,NaN,NaN
5,5,2023-09-13T03:11:07.307,/home/d/dagoret/rubin-user/DATA/AuxtelData2023...,exposure_2023091200282_dmpostisrccd.fits,282,HD009051,collimator,holo4_003,1.484045,30.0,...,-0.5500,73.437500,6.4218,NaN,NaN,NaN,NaN,,NaN,NaN
6,6,2023-09-13T03:19:59.235,/home/d/dagoret/rubin-user/DATA/AuxtelData2023...,exposure_2023091200296_dmpostisrccd.fits,296,HD2811,collimator,holo4_003,1.180937,30.0,...,-0.4750,68.125000,6.4218,NaN,NaN,NaN,NaN,,NaN,NaN
7,7,2023-09-13T03:20:42.256,/home/d/dagoret/rubin-user/DATA/AuxtelData2023...,exposure_2023091200297_dmpostisrccd.fits,297,HD2811,collimator,holo4_003,1.179290,30.0,...,-0.4750,67.625000,5.4210,NaN,NaN,NaN,NaN,,NaN,NaN
8,8,2023-09-13T03:30:54.110,/home/d/dagoret/rubin-user/DATA/AuxtelData2023...,exposure_2023091200311_dmpostisrccd.fits,311,HD205905,collimator,holo4_003,1.009810,30.0,...,-0.5250,69.550003,5.6712,NaN,NaN,NaN,NaN,,NaN,NaN
9,9,2023-09-13T03:31:37.086,/home/d/dagoret/rubin-user/DATA/AuxtelData2023...,exposure_2023091200312_dmpostisrccd.fits,312,HD205905,collimator,holo4_003,1.010173,30.0,...,-0.5500,68.574997,6.5052,NaN,NaN,NaN,NaN,,NaN,NaN


# Make logbook


In [47]:
N=len(df)
N

24

In [48]:
if N>0:
#    df.to_excel(logbook_xlsx)
    df.to_csv(logbook_csv)

In [49]:
logbook_csv

'logbooks/auxtellogbook_collimator~holo4_003_20230912_v7.0.csv'